In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
y_train = train['label']
y_train.shape
X_train = train.drop(labels=['label'], axis=1)

In [ ]:
X_train = X_train / 255
test = test / 255

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.model_selection import train_test_split
    
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [ ]:
y_train = to_categorical(y_train, num_classes=10)
print(y_train.shape)
print(X_train.shape)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=42)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
y_train[0]

In [ ]:
g = plt.imshow(X_train[0][:,:,0])

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
output_dir_path = '../output/'
if not os.path.exists(output_dir_path):
    os.mkdir(output_dir_path)
    
os.listdir('../')

In [ ]:
epochs = 10

# checkpointer = ModelCheckpoint(filepath='../input/saved_model.hdf5', verbose=1, save_best_only=True)
checkpointer = ModelCheckpoint('../output/{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, 
          callbacks=[checkpointer], batch_size=20, verbose=1)

In [ ]:
os.listdir(output_dir_path)

In [ ]:
def save_best_model(epoch, dir_path, num_ext, ext):
    tmp_file_name = os.listdir(dir_path)
    test = []
    num_element = -num_ext

    for x in range(0, len(tmp_file_name)):
        test.append(tmp_file_name[x][:num_element])
        float(test[x])

    highest = max(test)

    return str(highest) + ext

best = save_best_model(10, output_dir_path, 5, ".hdf5")

print(best)

In [ ]:
model.load_weights(os.path.join(output_dir_path, best))

In [ ]:
model.weights

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
results.shape

In [ ]:
output = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

output.to_csv("DigitRecognizerChallengeSub2.csv",index=False)

In [ ]:
os.listdir('.')